In [149]:
# Librerias
import pandas as pd
import numpy as np

In [150]:
# Cargar dataset de ejemplo
data = {
    "ID": [1, 2, 3, 4, 5, "6"],
    "Genero": ["Masculino", "F", "M", "Fem", "Femenino", np.nan],
    "Ciudad": ["Madrid", "Barcelona", "Madrid", "Bilbao", "Sevilla", "Bilbao"],
    "Edad": [15, 22, 34, 45, 58, 67],
    "Ingreso": [25000, 32000, 28000, np.nan, 50000, 54000]
}
df = pd.DataFrame(data)
df

,ID,Genero,Ciudad,Edad,Ingreso
0,1,Masculino,Madrid,15,25000.0
1,2,F,Barcelona,22,32000.0
2,3,M,Madrid,34,28000.0
3,4,Fem,Bilbao,45,NaN
4,5,Femenino,Sevilla,58,50000.0
5,6,NaN,Bilbao,67,54000.0


In [151]:
# 1️⃣ Asegurar que los tipos de datos sean correctos
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   ID       6 non-null      object 
 1   Genero   5 non-null      object 
 2   Ciudad   6 non-null      object 
 3   Edad     6 non-null      int64  
 4   Ingreso  5 non-null      float64
dtypes: float64(1), int64(1), object(3)
memory usage: 368.0+ bytes


In [152]:
#print(df[df['Genero'] == 'M'])
df['Genero'].unique()

array(['Masculino', 'F', 'M', 'Fem', 'Femenino', nan], dtype=object)

In [153]:
# Convertir Genero a categórica
df["Genero"] = df["Genero"].replace({
    "M": "Masculino",
    "F": "Femenino",
    "Fem": "Femenino"
})
df["Genero"] = df["Genero"].astype("category")

print("\nTipos de datos corregidos:\n", df.dtypes)



Tipos de datos corregidos:
 ID           object
Genero     category
Ciudad       object
Edad          int64
Ingreso     float64
dtype: object


In [154]:
# Convertir ID a integer
df["ID"] = pd.to_numeric(df["ID"], errors="coerce").astype("Int64")  # Int64 permite NaN

In [155]:
# 2️⃣ Manejo de valores nulos


In [156]:
# Y si quieres ver las filas que tienen muchos nulos??


# Ver las filas con más de X valores nulos (por ejemplo, más de 3 nulos)
filas_con_nulos = df[df.isnull().sum(axis=1) > 1]
print("\nFilas con más de 1 nulo:\n", filas_con_nulos)



Filas con más de 1 nulo:
 Empty DataFrame
Columns: [ID, Genero, Ciudad, Edad, Ingreso]
Index: []


In [157]:
# Imputar nulos
filas_con_nulos = df[df.isnull().sum(axis=1) > 1]
print("\nFilas con más de 1 nulo:\n", filas_con_nulos)

# Imputar numérico con mediana
for col in df.select_dtypes(include=["int64", "float64", "Int64"]).columns:
    mediana = df[col].median()
    df[col] = df[col].fillna(mediana)

# Imputar categórico con la moda u otro valor
for col in df.select_dtypes(include=["category", "object"]).columns:
    moda = df[col].mode()[0]
    df[col] = df[col].fillna(moda)


Filas con más de 1 nulo:
 Empty DataFrame
Columns: [ID, Genero, Ciudad, Edad, Ingreso]
Index: []


In [ ]:
# 3️⃣ Limpieza de valores categóricos  - con apply lambda

# a) Quédate sólo con la inicial del género
df['GeneroInicial'] = df['Genero'].apply(lambda x: str(x)[0] if pd.notnull(x) else np.nan)
print("Genero inicial:\n", df[['Genero', 'GeneroInicial']])

Genero inicial:
       Genero GeneroInicial
0  Masculino             M
1   Femenino             F
2  Masculino             M
3   Femenino             F
4   Femenino             F
5   Femenino             F


In [159]:
# 3️⃣ Limpieza de valores categóricos  - con map()

# a) Estandarizalo a Femenino, Masculino y Neutro
map_genero = {
    "M": "Masculino",
    "F": "Femenino",
    "Fem": "Femenino",
    "Masculino": "Masculino",
    "Femenino": "Femenino",
    np.nan: "Neutro"
}

df['GeneroEstandarizado'] = df['Genero'].map(lambda x: map_genero.get(x, "Neutro"))
print(df)

   ID     Genero     Ciudad  Edad  Ingreso GeneroInicial GeneroEstandarizado
0   1  Masculino     Madrid    15  25000.0             M           Masculino
1   2   Femenino  Barcelona    22  32000.0             F            Femenino
2   3  Masculino     Madrid    34  28000.0             M           Masculino
3   4   Femenino     Bilbao    45  32000.0             F            Femenino
4   5   Femenino    Sevilla    58  50000.0             F            Femenino
5   6   Femenino     Bilbao    67  54000.0             F            Femenino


In [160]:
print("\nGenero estandarizado:\n", df[['Genero', 'GeneroEstandarizado']])


Genero estandarizado:
       Genero GeneroEstandarizado
0  Masculino           Masculino
1   Femenino            Femenino
2  Masculino           Masculino
3   Femenino            Femenino
4   Femenino            Femenino
5   Femenino            Femenino


In [161]:
# 4️⃣ Reducir cardinalidad en variables categóricas: haz una lista de las ciudades mas frecuentes y quitate categorias

# Obtener las ciudades más frecuentes (ejemplo top 3)
top_ciudades = df['Ciudad'].value_counts().nlargest(3).index.tolist()
top_ciudades

['Madrid', 'Bilbao', 'Barcelona']

In [162]:
# Reemplazar el resto por 'Otros'
df['CiudadReducida'] = df['Ciudad'].apply(lambda x: x if x in top_ciudades else 'Otros')
print("\nCiudad con cardinalidad reducida:\n", df[['Ciudad', 'CiudadReducida']])


Ciudad con cardinalidad reducida:
       Ciudad CiudadReducida
0     Madrid         Madrid
1  Barcelona      Barcelona
2     Madrid         Madrid
3     Bilbao         Bilbao
4    Sevilla          Otros
5     Bilbao         Bilbao


In [163]:
# 5️⃣ Binning en variables numéricas (Ejemplo: Edad)

# Crear bins y etiquetas
bins = [0, 18, 35, 50, 100]
labels = ['Joven', 'Adulto', 'Maduro', 'Mayor']
df['EdadBinned'] = pd.cut(df['Edad'], bins=bins, labels=labels)
print("\nEdad binning:\n", df[['Edad', 'EdadBinned']])


Edad binning:
    Edad EdadBinned
0    15      Joven
1    22     Adulto
2    34     Adulto
3    45     Maduro
4    58      Mayor
5    67      Mayor
